In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv("IMDB_movies2.csv")
df.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director_spaces,director,writer,production_company,actors_spaces,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0004134,Hypocrites,Hypocrites,1915,1/20/1915,Drama,54,USA,English,Lois Weber,Lois Weber,Lois Weber,Hobart Bosworth Productions,"Courtenay Foote, Myrtle Stedman, Herbert Stand...","Courtenay Foote, Myrtle Stedman, Herbert Stand...",The parallel stories of a modern preacher and ...,6.6,429,NaN,NaN,NaN,NaN,41,8.0
1,tt0004707,Tillie's Punctured Romance,Tillie's Punctured Romance,1914,12/21/1914,Comedy,82,USA,English,"Mack Sennett, Charles Bennett","Mack Sennett, Charles Bennett",NaN,Keystone Film Company,"Marie Dressler, Charles Chaplin, Mabel Normand...","Marie Dressler, Charles Chaplin, Mabel Normand...",A con man from the city dupes a wealthy countr...,6.3,3164,"$50,000",NaN,NaN,NaN,37,18.0
2,tt0005078,The Cheat,The Cheat,1915,12/13/1915,"Drama, Romance",59,USA,English,Cecil B. DeMille,Cecil B. DeMille,"Hector Turnbull, Jeanie Macpherson",Jesse L. Lasky Feature Play Company,"Fannie Ward, Sessue Hayakawa, Jack Dean, James...","Fannie Ward, Sessue Hayakawa, Jack Dean, James...","A venal, spoiled stockbroker's wife impulsivel...",6.6,2217,"$17,311",NaN,NaN,NaN,38,19.0
3,tt0006333,"20,000 Leagues Under the Sea","20,000 Leagues Under the Sea",1916,12/24/1916,"Action, Adventure, Sci-Fi",105,USA,English,Stuart Paton,Stuart Paton,Jules Verne,Williamson Submarine Film Corporation,"Dan Hanlon, Edna Pendleton, Curtis Benton, All...","Dan Hanlon, Edna Pendleton, Curtis Benton, All...",A French professor and his daughter accompany ...,6.2,1440,"$200,000",NaN,NaN,NaN,26,16.0
4,tt0009611,Shoulder Arms,Shoulder Arms,1918,10/27/1918,"Comedy, War",45,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Edna Purviance, Charles Chaplin, Syd Chaplin, ...","Edna Purviance, Charles Chaplin, Syd Chaplin, ...",Charlie is a boot camp private who has a dream...,7.3,5411,NaN,NaN,NaN,NaN,35,21.0


If you visualize the dataset, you will see that it has many extra info about a movie. We don’t need all of them. So, we choose keywords, cast, genres and director column to use as our feature set(the so called “content” of the movie).

In [6]:
df = df[["title", "year", "genre", "country", "director", "writer", "production_company", "actors", "description", "avg_vote", "actors_spaces", "director_spaces"]]
df.head()

,title,year,genre,country,director,writer,production_company,actors,description,avg_vote,actors_spaces,director_spaces
0,Hypocrites,1915,Drama,USA,Lois Weber,Lois Weber,Hobart Bosworth Productions,"Courtenay Foote, Myrtle Stedman, Herbert Stand...",The parallel stories of a modern preacher and ...,6.6,"Courtenay Foote, Myrtle Stedman, Herbert Stand...",Lois Weber
1,Tillie's Punctured Romance,1914,Comedy,USA,"Mack Sennett, Charles Bennett",NaN,Keystone Film Company,"Marie Dressler, Charles Chaplin, Mabel Normand...",A con man from the city dupes a wealthy countr...,6.3,"Marie Dressler, Charles Chaplin, Mabel Normand...","Mack Sennett, Charles Bennett"
2,The Cheat,1915,"Drama, Romance",USA,Cecil B. DeMille,"Hector Turnbull, Jeanie Macpherson",Jesse L. Lasky Feature Play Company,"Fannie Ward, Sessue Hayakawa, Jack Dean, James...","A venal, spoiled stockbroker's wife impulsivel...",6.6,"Fannie Ward, Sessue Hayakawa, Jack Dean, James...",Cecil B. DeMille
3,"20,000 Leagues Under the Sea",1916,"Action, Adventure, Sci-Fi",USA,Stuart Paton,Jules Verne,Williamson Submarine Film Corporation,"Dan Hanlon, Edna Pendleton, Curtis Benton, All...",A French professor and his daughter accompany ...,6.2,"Dan Hanlon, Edna Pendleton, Curtis Benton, All...",Stuart Paton
4,Shoulder Arms,1918,"Comedy, War",USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Edna Purviance, Charles Chaplin, Syd Chaplin, ...",Charlie is a boot camp private who has a dream...,7.3,"Edna Purviance, Charles Chaplin, Syd Chaplin, ...",Charles Chaplin


In [7]:
df['avg_vote'] = df['avg_vote'].astype(str)
df['year'] = df['year'].astype(str)
df['description'] = df['description'].astype(str)

In [8]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop.append('A')
stop.append('The')
df['keywords'] = df['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df.head()

,title,year,genre,country,director,writer,production_company,actors,description,avg_vote,actors_spaces,director_spaces,keywords
0,Hypocrites,1915,Drama,USA,Lois Weber,Lois Weber,Hobart Bosworth Productions,"Courtenay Foote, Myrtle Stedman, Herbert Stand...",The parallel stories of a modern preacher and ...,6.6,"Courtenay Foote, Myrtle Stedman, Herbert Stand...",Lois Weber,parallel stories modern preacher medieval monk...
1,Tillie's Punctured Romance,1914,Comedy,USA,"Mack Sennett, Charles Bennett",NaN,Keystone Film Company,"Marie Dressler, Charles Chaplin, Mabel Normand...",A con man from the city dupes a wealthy countr...,6.3,"Marie Dressler, Charles Chaplin, Mabel Normand...","Mack Sennett, Charles Bennett",con man city dupes wealthy country girl marriage.
2,The Cheat,1915,"Drama, Romance",USA,Cecil B. DeMille,"Hector Turnbull, Jeanie Macpherson",Jesse L. Lasky Feature Play Company,"Fannie Ward, Sessue Hayakawa, Jack Dean, James...","A venal, spoiled stockbroker's wife impulsivel...",6.6,"Fannie Ward, Sessue Hayakawa, Jack Dean, James...",Cecil B. DeMille,"venal, spoiled stockbroker's wife impulsively ..."
3,"20,000 Leagues Under the Sea",1916,"Action, Adventure, Sci-Fi",USA,Stuart Paton,Jules Verne,Williamson Submarine Film Corporation,"Dan Hanlon, Edna Pendleton, Curtis Benton, All...",A French professor and his daughter accompany ...,6.2,"Dan Hanlon, Edna Pendleton, Curtis Benton, All...",Stuart Paton,French professor daughter accompany Captain Ne...
4,Shoulder Arms,1918,"Comedy, War",USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Edna Purviance, Charles Chaplin, Syd Chaplin, ...",Charlie is a boot camp private who has a dream...,7.3,"Edna Purviance, Charles Chaplin, Syd Chaplin, ...",Charles Chaplin,Charlie boot camp private dream hero goes dari...


In [10]:
df['title_year'] = df['title'] + ' (' + df['year'].astype(str) + ')'
df.head()

,title,year,genre,country,director,writer,production_company,actors,description,avg_vote,actors_spaces,director_spaces,keywords,title_year
0,Hypocrites,1915,Drama,USA,Lois Weber,Lois Weber,Hobart Bosworth Productions,"Courtenay Foote, Myrtle Stedman, Herbert Stand...",The parallel stories of a modern preacher and ...,6.6,"Courtenay Foote, Myrtle Stedman, Herbert Stand...",Lois Weber,parallel stories modern preacher medieval monk...,Hypocrites (1915)
1,Tillie's Punctured Romance,1914,Comedy,USA,"Mack Sennett, Charles Bennett",NaN,Keystone Film Company,"Marie Dressler, Charles Chaplin, Mabel Normand...",A con man from the city dupes a wealthy countr...,6.3,"Marie Dressler, Charles Chaplin, Mabel Normand...","Mack Sennett, Charles Bennett",con man city dupes wealthy country girl marriage.,Tillie's Punctured Romance (1914)
2,The Cheat,1915,"Drama, Romance",USA,Cecil B. DeMille,"Hector Turnbull, Jeanie Macpherson",Jesse L. Lasky Feature Play Company,"Fannie Ward, Sessue Hayakawa, Jack Dean, James...","A venal, spoiled stockbroker's wife impulsivel...",6.6,"Fannie Ward, Sessue Hayakawa, Jack Dean, James...",Cecil B. DeMille,"venal, spoiled stockbroker's wife impulsively ...",The Cheat (1915)
3,"20,000 Leagues Under the Sea",1916,"Action, Adventure, Sci-Fi",USA,Stuart Paton,Jules Verne,Williamson Submarine Film Corporation,"Dan Hanlon, Edna Pendleton, Curtis Benton, All...",A French professor and his daughter accompany ...,6.2,"Dan Hanlon, Edna Pendleton, Curtis Benton, All...",Stuart Paton,French professor daughter accompany Captain Ne...,"20,000 Leagues Under the Sea (1916)"
4,Shoulder Arms,1918,"Comedy, War",USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Edna Purviance, Charles Chaplin, Syd Chaplin, ...",Charlie is a boot camp private who has a dream...,7.3,"Edna Purviance, Charles Chaplin, Syd Chaplin, ...",Charles Chaplin,Charlie boot camp private dream hero goes dari...,Shoulder Arms (1918)


In [11]:
df.dtypes

title                 object
year                  object
genre                 object
country               object
director              object
writer                object
production_company    object
actors                object
description           object
avg_vote              object
actors_spaces         object
director_spaces       object
keywords              object
title_year            object
dtype: object

In [12]:
for i in ["director","writer", "actors", "production_company"]:
    df[i] = df[i].str.replace(' ', '')

In [13]:
for i in ["director","writer", "actors", "production_company", "genre"]:
    df[i] = df[i].str.replace(',', ' ')

In [14]:
features = ['title', 'director', 'year', 'genre', 'keywords', 'production_company', 'writer', 'actors']

Our next task is to create a function for combining the values of these columns into a single string.

In [15]:
def combine_features(row):
    return row['title']+" "+((row['director'] +" ") * 2)+" "+row['year']+" "+((row['genre'] +" ") * 2)+" "+row['keywords']+" "+row['production_company']+" "+row['writer']+" "+row['actors']

Now, we need to call this function over each row of our dataframe. But, before doing that, we need to clean and preprocess the data for our use. We will fill all the NaN values with blank string in the dataframe.

In [16]:
for feature in features:
    df[feature] = df[feature].fillna('') #filling all NaNs with blank string

df["combined_features"] = df.apply(combine_features,axis=1) #applying combined_features() method over each rows of dataframe and storing the combined string in "combined_features" column


In [19]:
df.iloc[3].combined_features

'20,000 Leagues Under the Sea StuartPaton StuartPaton  1916 Action  Adventure  Sci-Fi Action  Adventure  Sci-Fi  French professor daughter accompany Captain Nemo adventure aboard submarine. WilliamsonSubmarineFilmCorporation JulesVerne DanHanlon EdnaPendleton CurtisBenton AllenHolubar MattMoore JaneGail HowardCrampton WilliamWelsh LoisAlexander WallisClark'

In [ ]:
df.to_csv("IMDB_movies_clean.csv")

Now that we have obtained the combined strings, we can now feed these strings to a CountVectorizer() object for getting the count matrix.

In [20]:
cv = CountVectorizer() #creating new CountVectorizer() object
count_matrix = cv.fit_transform(df["combined_features"]) #feeding combined strings(movie contents) to CountVectorizer() object
count_matrix

<14067x128356 sparse matrix of type '<class 'numpy.int64'>'
	with 556685 stored elements in Compressed Sparse Row format>

At this point, 60% work is done. Now, we need to obtain the cosine similarity matrix from the count matrix.

In [21]:
cosine_sim = cosine_similarity(count_matrix)

Now, we will define two helper functions to get movie title from movie index and vice-versa.

In [22]:
df = df.reset_index()
df.head()

,index,title,year,genre,country,director,writer,production_company,actors,description,avg_vote,actors_spaces,director_spaces,keywords,title_year,combined_features
0,0,Hypocrites,1915,Drama,USA,LoisWeber,LoisWeber,HobartBosworthProductions,CourtenayFoote MyrtleStedman HerbertStanding A...,The parallel stories of a modern preacher and ...,6.6,"Courtenay Foote, Myrtle Stedman, Herbert Stand...",Lois Weber,parallel stories modern preacher medieval monk...,Hypocrites (1915),Hypocrites LoisWeber LoisWeber 1915 Drama Dra...
1,1,Tillie's Punctured Romance,1914,Comedy,USA,MackSennett CharlesBennett,,KeystoneFilmCompany,MarieDressler CharlesChaplin MabelNormand Mack...,A con man from the city dupes a wealthy countr...,6.3,"Marie Dressler, Charles Chaplin, Mabel Normand...","Mack Sennett, Charles Bennett",con man city dupes wealthy country girl marriage.,Tillie's Punctured Romance (1914),Tillie's Punctured Romance MackSennett Charles...
2,2,The Cheat,1915,Drama Romance,USA,CecilB.DeMille,HectorTurnbull JeanieMacpherson,JesseL.LaskyFeaturePlayCompany,FannieWard SessueHayakawa JackDean JamesNeill ...,"A venal, spoiled stockbroker's wife impulsivel...",6.6,"Fannie Ward, Sessue Hayakawa, Jack Dean, James...",Cecil B. DeMille,"venal, spoiled stockbroker's wife impulsively ...",The Cheat (1915),The Cheat CecilB.DeMille CecilB.DeMille 1915 ...
3,3,"20,000 Leagues Under the Sea",1916,Action Adventure Sci-Fi,USA,StuartPaton,JulesVerne,WilliamsonSubmarineFilmCorporation,DanHanlon EdnaPendleton CurtisBenton AllenHolu...,A French professor and his daughter accompany ...,6.2,"Dan Hanlon, Edna Pendleton, Curtis Benton, All...",Stuart Paton,French professor daughter accompany Captain Ne...,"20,000 Leagues Under the Sea (1916)","20,000 Leagues Under the Sea StuartPaton Stuar..."
4,4,Shoulder Arms,1918,Comedy War,USA,CharlesChaplin,CharlesChaplin,CharlesChaplinProductions,EdnaPurviance CharlesChaplin SydChaplin LoyalU...,Charlie is a boot camp private who has a dream...,7.3,"Edna Purviance, Charles Chaplin, Syd Chaplin, ...",Charles Chaplin,Charlie boot camp private dream hero goes dari...,Shoulder Arms (1918),Shoulder Arms CharlesChaplin CharlesChaplin 1...


In [23]:
def get_title_from_index(index):
    return df[df.index == index]["title_year"].values[0]
def get_index_from_title(title_year):
    return df[df.title_year == title_year]["index"].values[0]

Our next step is to get the title of the movie that the user currently likes. Then we will find the index of that movie. After that, we will access the row corresponding to this movie in the similarity matrix. Thus, we will get the similarity scores of all other movies from the current movie. Then we will enumerate through all the similarity scores of that movie to make a tuple of movie index and similarity score. This will convert a row of similarity scores like this- `[1 0.5 0.2 0.9]` to this- `[(0, 1) (1, 0.5) (2, 0.2) (3, 0.9)]` . Here, each item is in this form- (movie index, similarity score).

In [24]:
movie_user_likes = input("Enter Movie Title: ") 

Enter Movie Title: Muppets Most Wanted (2014)


In [25]:
#movie_user_likes = "Bedknobs and Broomsticks"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index])) #accessing the row corresponding to given movie to find all the similarity scores for that movie and then enumerating over it


Now comes the most vital point. We will sort the list `similar_movies` according to similarity scores in descending order. Since the most similar movie to a given movie will be itself, we will discard the first element after sorting the movies.

In [26]:
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

Now, we will run a loop to print first 5 entries from `sorted_similar_movies` list.

In [27]:
i=0
print("Top 6 similar movies to "+movie_user_likes+" are:\n")
for element in sorted_similar_movies:
    print(get_title_from_index(element[0]))
    i=i+1
    if i>5:
        break

Top 6 similar movies to Muppets Most Wanted (2014) are:

The Muppets (2011)
Muppet Treasure Island (1996)
The Grand Budapest Hotel (2014)
Topkapi (1964)
The Last Guest (2017)
The Great Muppet Caper (1981)
